In [17]:
#Convertir OFF a BINVOX

In [18]:
import os
import glob
import importlib
import numpy as np
import scipy.ndimage
import binvox_rw

try:
    importlib.import_module('trimesh')
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'trimesh'])
import trimesh

In [13]:
data = 'ModelNet10'

target = {'bathtub', 'chair', 'dresser', 'night_stand', 'sofa', 'toilet', 'bed', 'desk', 'monitor', 'table'}

for c in target:
    for split in ['test', 'train']:
        for off in glob.glob(os.path.join(data, c, split, '*.off')):
            print("In Progress ...")
            voxname = os.path.join(data, c, split, os.path.basename(off).split('.')[0] + '.binvox')
            if os.path.exists(voxname):
                print(voxname, "generating...")
                continue
            os.system(f'binvox.exe -d 32 -cb -e {off}')

In Process ...
ModelNet10\toilet\test\toilet_0345.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0346.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0347.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0348.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0349.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0350.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0351.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0352.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0353.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0354.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0355.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0356.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0357.binvox generating...
In Process ...
ModelNet10\toilet\test\toilet_0358.binvox generating...
In Pro

KeyboardInterrupt: 

In [ ]:
## Generar agujeros
## Recibe archivos .binvox para convertirlos en array, editarlos, y volver a pasarlos a .binvox.

In [14]:
def get_fractured(source, min_points=1, max_points=3, min_radius=7, max_radius=10, sphere_chance=0.85):
    fractured = source.copy()

    # Escoger cantidad aleatoria de agujeros
    points = np.random.randint(min_points, max_points+1)

    # Todos los indices donde existan voxels
    idxs = np.argwhere(fractured == True) 

    # Escoger donde se encontraran los centros de los huecos
    centers = idxs[np.random.choice(len(idxs), points, False)]
    
    for x,y,z in centers:
        # Radio del hueco aleatorio
        r = np.random.randint(min_radius, max_radius+1) 
        xmin, xmax = max(0, x-r), x+r
        ymin, ymax = max(0, y-r), y+r
        zmin, zmax = max(0, z-r), z+r
        hole = np.ones_like(fractured)
        # Establecer los valores de los elementos del hueco
        hole[xmin:xmax, ymin:ymax, zmin:zmax] = False

        # Convertir la forma del hueco a una esfera si se cumple la probabilidad
        if np.random.rand() < sphere_chance:
            idxs = np.argwhere(hole == False)
            idxs_remove = np.sqrt((idxs[:,0] - x)**2 + (idxs[:,1] - y)**2 + (idxs[:,2] - z)**2)
            idxs_remove = idxs[idxs_remove > r]
            hole[idxs_remove[:,0], idxs_remove[:,1], idxs_remove[:,2]] = True
            
        # Crear agujero en fractured
        fractured *= hole
    
    return fractured


In [16]:
#@title Read  .binvox
# Abrir modelo .binvox y convertirlo a un array

for c in target:
    for split in ['test', 'train']:
        for binx in glob.glob(os.path.join(dataset, c, split, '*.binvox')):
            voxelname = os.path.join(dataset, c, split, os.path.basename(binx))
            if os.path.exists(voxelname):
                with open(voxelname, 'rb') as f:
                    model = binvox_rw.read_as_3d_array(f)
                    # Fracturar el modelo con los valores por defecto
                    fractured_model = get_fractured(model.data)
                    # Convertir geometrix voxelizada en una malla de triangulos usando el algoritmo Marching Cubes
                    mesh = trimesh.voxel.ops.matrix_to_marching_cubes(matrix=fractured_model, pitch=1.0)
                    
                    # Preparar y limiar la malla resultante
                    mesh.merge_vertices()
                    mesh.remove_duplicate_faces()
                    mesh.apply_scale(scaling=1.0)
                    
                    # Corregir normales invertidas en la malla
                    trimesh.repair.fix_inversion(mesh)
                    # Ajustar la orientacion de las caras de la malla
                    trimesh.repair.fix_winding(mesh)
                    modelname=os.path.basename(voxelname).split('.')[0]
                    print(f'ModelNet10/{c}/{split}/{modelname}.stl')
                    mesh.export(f'ModelNet10/{c}/{split}/{modelname}.stl', file_type='stl')

ModelNet10/toilet/test/toilet_0345.stl
ModelNet10/toilet/test/toilet_0346.stl
ModelNet10/toilet/test/toilet_0347.stl
ModelNet10/toilet/test/toilet_0348.stl
ModelNet10/toilet/test/toilet_0349.stl
ModelNet10/toilet/test/toilet_0350.stl
ModelNet10/toilet/test/toilet_0351.stl
ModelNet10/toilet/test/toilet_0352.stl
ModelNet10/toilet/test/toilet_0353.stl
ModelNet10/toilet/test/toilet_0354.stl
ModelNet10/toilet/test/toilet_0355.stl
ModelNet10/toilet/test/toilet_0356.stl
ModelNet10/toilet/test/toilet_0357.stl
ModelNet10/toilet/test/toilet_0358.stl
ModelNet10/toilet/test/toilet_0359.stl
ModelNet10/toilet/test/toilet_0360.stl
ModelNet10/toilet/test/toilet_0361.stl
ModelNet10/toilet/test/toilet_0362.stl
ModelNet10/toilet/test/toilet_0363.stl
ModelNet10/toilet/test/toilet_0364.stl
ModelNet10/toilet/test/toilet_0365.stl
ModelNet10/toilet/test/toilet_0366.stl
ModelNet10/toilet/test/toilet_0367.stl
ModelNet10/toilet/test/toilet_0368.stl
ModelNet10/toilet/test/toilet_0369.stl
ModelNet10/toilet/test/to

KeyboardInterrupt: 